## Build Functions for ELT

In [6]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [7]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Download, Transform, and Modeling All in One

In [8]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [11]:
f.predictions('brk-b')

BRK-B 5m Interval Timestamp: 2024-11-24 04:07:18 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,down
kelly_1:2.5,0.228009,0.233784,0.267692,0.156725
prob_up,0.269948,0.253914,0.208389,0.25
prob_static,0.230876,0.315572,0.325009,0.25
prob_down,0.499176,0.430515,0.466603,0.5
precision,0.448578,0.452703,0.476923,0.397661
recall,0.412475,0.404427,0.43662,0.273642
f1,0.429769,0.427205,0.455882,0.324195
support,"[497.0, 494.0, 497.0]","[497.0, 494.0, 497.0]","[497.0, 494.0, 497.0]","[497.0, 494.0, 497.0]"


BRK-B 15m Interval Timestamp: 2024-11-24 04:07:26 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.155882,0.110692,0.152113,0.039744
prob_up,0.385416,0.398773,0.366588,0.375
prob_static,0.324812,0.307521,0.350857,0.25
prob_down,0.289772,0.293706,0.282555,0.375
precision,0.397059,0.36478,0.394366,0.314103
recall,0.327273,0.351515,0.339394,0.29697
f1,0.358804,0.358025,0.364821,0.305296
support,"[165.0, 167.0, 167.0]","[165.0, 167.0, 167.0]","[165.0, 167.0, 167.0]","[165.0, 167.0, 167.0]"


BRK-B 1h Interval Timestamp: 2024-11-24 04:07:29 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,static,up,down
kelly_1:2.5,0.101717,0.094974,0.089849,0.033628
prob_up,0.379312,0.315857,0.355867,0.125
prob_static,0.273571,0.347314,0.315161,0.375
prob_down,0.347117,0.336829,0.328972,0.5
precision,0.358369,0.353553,0.349892,0.309735
recall,0.350105,0.425887,0.339623,0.221053
f1,0.354189,0.386364,0.344681,0.257985
support,"[477.0, 479.0, 475.0]","[477.0, 479.0, 475.0]","[477.0, 479.0, 475.0]","[477.0, 479.0, 475.0]"


BRK-B 1wk Interval Timestamp: 2024-11-24 04:07:43 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,static,down,static
kelly_1:2.5,0.08932,0.070796,0.12,0.156164
prob_up,0.218332,0.31174,0.297939,0.375
prob_static,0.389569,0.358978,0.316658,0.375
prob_down,0.3921,0.329282,0.385403,0.25
precision,0.349515,0.336283,0.371429,0.39726
recall,0.371134,0.387755,0.402062,0.591837
f1,0.36,0.36019,0.386139,0.47541
support,"[98.0, 98.0, 97.0]","[98.0, 98.0, 97.0]","[98.0, 98.0, 97.0]","[98.0, 98.0, 97.0]"


BRK-B 1mo Interval Timestamp: 2024-11-24 04:07:45 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,down,down
kelly_1:2.5,0.125,0.125,0.23,0.272
prob_up,0.121744,0.219906,0.250858,0.125
prob_static,0.451813,0.527917,0.363187,0.25
prob_down,0.426443,0.252178,0.385955,0.625
precision,0.375,0.375,0.45,0.48
recall,0.3,0.3,0.409091,0.545455
f1,0.333333,0.333333,0.428571,0.510638
support,"[20.0, 20.0, 22.0]","[20.0, 20.0, 22.0]","[20.0, 20.0, 22.0]","[20.0, 20.0, 22.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')